In [1]:
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional

from random import randint
from NLP_little_helpers import *
little_helpers()

Using TensorFlow backend.
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, predict_tags, tagged_n_grams, unknown_words_X, check_and_predict


In [2]:
with open('quotes-100-en.json', encoding="utf8") as file:
    dct_text = json.load(file)

In [3]:
text = clean_data([q for person, quotes in dct_text.items() for q in quotes])
print("Total of {} quotes.".format(len(text)))

Total of 41538 quotes.


In [15]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Total of {} unique words.".format(vocab_size))
 
# separate into input(X) and output(y)
sequences = np.array(pad_sequences(sequences))
X, y = sequences[:,:-2], sequences[:,-2] # has to be -2, since -1 gives back the <eos> tag
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]


Total of 31325 unique words.


In [18]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=seq_length))
model.add(Bidirectional(LSTM(16)))#, return_sequences=True)))
#model.add(LSTM(16))
#model.add(Dense(8, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 28, 32)            1002400   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                6272      
_________________________________________________________________
dense_2 (Dense)              (None, 31325)             1033725   
Total params: 2,042,397
Trainable params: 2,042,397
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=1, shuffle=True, validation_split=0.2)
 # save the model to file
model.save('sequencing_model.h5')

C:\Users\felip\Miniconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 33230 samples, validate on 8308 samples
Epoch 1/1
33230/33230 [==============================] - 177s 5ms/step - loss: 9.0377 - accuracy: 0.0224 - val_loss: 8.5668 - val_accuracy: 0.0199


In [21]:
seed_text = text[randint(0,len(text))]
print(seed_text + '\n')

encoded = tokenizer.texts_to_sequences([seed_text])[0]

# predict probabilities for each word
y_hat = model.predict_classes(encoded, verbose=0)


seq2word_map = dict(map(reversed, tokenizer.word_index.items()))

out_word = seq2word_map[y_hat]
        
encoded = pad_sequences([encoded], maxlen=seq_length)
encoded

<bos> a healthy ecology is the basis for a healthy economy <eos>



ValueError: Error when checking input: expected embedding_2_input to have shape (28,) but got array with shape (1,)

In [ ]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [ ]:
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

In [22]:
encoded

[1, 5, 1617, 12222, 4, 3, 1175, 17, 5, 1617, 1367, 2]

In [ ]:
# to do repartir em n-grams para adivinhar a proxima